# Machine Learning With Flowers

In [35]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [36]:
data = []

for dir, x, files in os.walk("flower_photos"):
    label = dir.split('/')[-1]
    for file in files:
        path = os.path.join(dir, file)
        data.append([path, label])

df = pd.DataFrame(data, columns=["file_path", "label"])

print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [13]:
datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [44]:
train_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = 'file_path',
    y_col = 'label',
    target_size = (224, 224),
    color_mode = 'rgb', 
    batch_size = 32,
    class_mode = 'categorical', 
    shuffle = True,
    subset = 'training'
)

valid_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = 'file_path',
    y_col = 'label',
    target_size = (224, 224),
    color_mode = 'rgb', 
    batch_size = 32,
    class_mode = 'categorical', 
    shuffle = True,
    subset = 'validation'
)

print(train_gen.class_indices.items())

Found 2936 validated image filenames belonging to 5 classes.
Found 734 validated image filenames belonging to 5 classes.
dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])


In [38]:

base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)

In [39]:
early_stopping = EarlyStopping(
    patience=10,
    min_delta=0,
    monitor='val_loss',
    verbose=0,
    restore_best_weights=True,
    baseline=None
)

In [41]:
model.fit(
    train_gen, 
    validation_data = valid_gen,
    epochs = 5,
    validation_steps=None,
    shuffle=False, 
    callbacks=early_stopping, 
    verbose=1
)

Epoch 1/5


/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 316s 3s/step - acc: 0.4027 - auc_3: 0.7106 - loss: 1.6518 - precision_3: 0.4568 - recall_3: 0.3059 - val_acc: 0.0954 - val_auc_3: 0.3375 - val_loss: 3.1564 - val_precision_3: 0.1186 - val_recall_3: 0.0790
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 281s 3s/step - acc: 0.7341 - auc_3: 0.9339 - loss: 0.7278 - precision_3: 0.7901 - recall_3: 0.6775 - val_acc: 0.1076 - val_auc_3: 0.3404 - val_loss: 3.9860 - val_precision_3: 0.1218 - val_recall_3: 0.1035
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 277s 3s/step - acc: 0.8367 - auc_3: 0.9718 - loss: 0.4665 - precision_3: 0.8733 - recall_3: 0.8021 - val_acc: 0.1144 - val_auc_3: 0.3373 - val_loss: 4.6160 - val_precision_3: 0.1197 - val_recall_3: 0.1076
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 283s 3s/step - acc: 0.8618 - auc_3: 0.9812 - loss: 0.3735 - precision_3: 0.8918 - recall_3: 0.8310 - val_acc: 0.1144 - val_auc_3: 0.3436 - val_loss: 4.9992 - val_precision_3: 0.1213 - val_recall_3: 0.1131
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 269s 

In [42]:
model.save('flowers.keras')

In [47]:
import cv2
from tensorflow.keras.saving import load_model
import numpy as np

model = load_model("flowers.keras")

input_image = cv2.imread('flower_photos/roses/11694025703_9a906fedc1_n.jpg')
input_image_resize = cv2.resize(input_image, (224,224))

input_image_scaled = input_image_resize/225
image_reshaped = np.reshape(input_image_scaled, [1,224,224,3])

print(train_gen.class_indices.items())

input_prediction = model.predict(image_reshaped)
print("Your flower is a :", np.argmax(input_prediction))

dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Your flower is a : 3
